# Bidding Models 

In [2]:
# Importing libaries & datasets 

import sys
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Other datasets
validate_data = pd.read_csv("datasets/validation.csv")

print(validate_data.head())

   click  weekday  hour                                     bidid  \
0      0        4    20  bbcb813b6166538503d8b33a5602d7d72f6019dc   
1      0        1    21  5a07316c49477cb5d9b4d5aa39c27d6c3be7f92d   
2      0        4     8  f6ece71dae81d6b16bfb24ad6dd5611472d4c673   
3      0        5    15  b4d5c57c9b38ff5a12954fa01e11931b4e6bfbbb   
4      0        1    18  0899bf144249458ea9c89188473694bf44c7ca15   

                                     userid       useragent             IP  \
0  663169f66491c98c69f1f94a5c48fa34aa9fe06f      windows_ie  211.144.203.*   
1  a23f0cfab6592c137f796e68fa752fceba08bb1b  windows_chrome   58.247.250.*   
2  5105a013ea54a9706146033e6a138d5234a3803e      windows_ie   117.12.111.*   
3  577e9a768ce17e73c61ba7022db927f9ee761830  windows_chrome   61.187.224.*   
4  e4bf79c562745d671b19f2edf7fda89c2e25987f  windows_chrome   117.41.145.*   

   region  city  adexchange                       ...                         \
0      79    79         1.0         

## Constant Bidding 

In [ ]:
def constant_bidding(const):
    impressions = 0
    clicks = 0
    cost = 0
    budget = 6250000

    for index, row in validate_data.iterrows():
        # Don't exceed budget
        if cost >= budget:
        # print("Elapsed budget")
            break
            
        if const > row['payprice']:
            impressions += 1
            clicks += row['click']
            cost += row['payprice']


    return impressions, cost, clicks

num_valid_impressions = validate_data.shape[0]

min_value = validate_data['payprice'].min()
if min_value % 2 == 1:
    min_value = min_value - 1

max_value = validate_data['bidprice'].max()

results = pd.DataFrame()
results['constants'] = np.arange(min_value, max_value, 2)

impressions = []
cost = []
clicks = []

for const in results['constants']:
    print(const)
    const_impressions, const_cost, const_clicks = constant_bidding(const)
    impressions.append(const_impressions)
    cost.append(const_cost)
    clicks.append(const_clicks)

results['impressions'] = impressions
results['cost'] = cost
results['clicks'] = clicks

total_num_clicks = len(validate_data.groupby('click').get_group(1))

results['CTR'] = (results['clicks']/results['impressions'])*100
results['CVR'] = (results['clicks']/total_num_clicks)*100
results['CPM'] = (results['cost']/results['impressions'])
results['eCPC'] = (results['cost']/results['clicks'])*100 

print(results)
#results.to_csv("output/constant_bid_output.csv")

0
2
4
6


## Random Bidding

In [ ]:
def random_bidding(min_bound, max_bound):
    impressions = 0
    cost = 0
    clicks = 0
    budget = 6250000

    for index, row in validate_data.iterrows():
        bid = randint(min_bound, max_bound)

        # Don't exceed budget
        if cost >= budget:
        # print("Elapsed budget")
            break
        if bid > row['payprice']:
            impressions += 1
            clicks += row['click']
            cost += row['payprice']

    return impressions, cost, clicks

num_valid_impressions = validate_data.shape[0]

min_value = validate_data['payprice'].min()
if min_value % 2 == 1:
    min_value = min_value - 1

max_value = validate_data['bidprice'].max()

results = pd.DataFrame()

results['constants'] = np.arange(min_value, max_value, 2)

impressions = []
cost = []
clicks = []

for const in results['constants']:
    print(const)
    const_impressions, const_cost, const_clicks = random_bidding(min_value, const)
    impressions.append(const_impressions)
    cost.append(const_cost)
    clicks.append(const_clicks)

results['impressions'] = impressions
results['cost'] = cost
results['clicks'] = clicks

total_num_clicks = len(validate_data.groupby('click').get_group(1))

results['CTR'] = (results['clicks']/results['impressions'])*100
results['CVR'] = (results['clicks']/total_num_clicks)*100
results['CPM'] = (results['cost']/results['impressions'])
results['eCPC'] = (results['cost']/results['clicks'])*100 

print(results)
#results.to_csv("random_bid_output.csv")